In [1]:
import onnx
from scc4onnx import order_conversion
from onnxsim import simplify
# from onnx_tf.backend import prepare

In [2]:
import onnx
import tf2onnx
import tensorflow as tf

In [3]:
onnx_model = onnx.load("yolov8n.onnx")
onnx_model.graph.input[0]

name: "images"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 640
      }
      dim {
        dim_value: 640
      }
    }
  }
}

In [4]:
input_name = onnx_model.graph.input[0].name
input_name

'images'

In [5]:
onnx_model.graph.input[0]

name: "images"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 640
      }
      dim {
        dim_value: 640
      }
    }
  }
}

In [6]:
onnx_model, check = simplify(onnx_model)
check

True

In [7]:
# %pip install onnx-tf
# %pip install tensorflow_probability
# %pip install --upgrade tensorflow


In [8]:
from onnx_tf.backend import prepare
tf_rep = prepare(onnx_model)

e:\downloads\yolo_py\.venv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [9]:
from pathlib import Path
data_dir = 'E:\downloads\yolo_py'
onnx_files = [p for p in Path(data_dir).iterdir() if p.suffix == '.onnx']
for onnx_file in onnx_files:
    # Load the ONNX model
    onnx_model = onnx.load(onnx_file)

    # Convert ONNX model to TensorFlow
    spec = (tf.TensorSpec((None, 640, 640, 3), tf.float32, name="input"),)
    # tf_model, _ = tf2onnx.convert.from_onnx_model(onnx_model, input_signature=spec)
    tf_rep = prepare(onnx_model)

    # Save the TensorFlow model
    tf_model_dir = onnx_file.stem + "_tf"
    

In [10]:
tf_model_dir

'yolov8n_tf'

In [11]:
tf_rep.export_graph(tf_model_dir)

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to onnx_tf_prefix__model_22_add_1_x, onnx_tf_prefix__model_22_mul_2_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: yolov8n_tf\assets


INFO:tensorflow:Assets written to: yolov8n_tf\assets
INFO:absl:Writing fingerprint to yolov8n_tf\fingerprint.pb


In [12]:
from IPython.display import Markdown, display

In [13]:
tfjs_model_dir = f"{tf_model_dir}-tfjs-uint8"
tfjs_model_dir

'yolov8n_tf-tfjs-uint8'

In [14]:
tfjs_convert_command = f"""tensorflowjs_converter
                 --input_format=tf_saved_model 
                 --output_format=tfjs_graph_model 
                 --signature_name=serving_default 
                 --saved_model_tags=serve 
                 "{tf_model_dir}" 
                 "{tfjs_model_dir}"
                 "--quantize_uint8"
                 """
tfjs_convert_command = " ".join(tfjs_convert_command.split())
display(Markdown(f"```bash\n{tfjs_convert_command}\n```"))

```bash
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve "yolov8n_tf" "yolov8n_tf-tfjs-uint8" "--quantize_uint8"
```

In [17]:
tf_rep.export_graph("yolov8n.tf")

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to onnx_tf_prefix__model_22_add_1_x, onnx_tf_prefix__model_22_mul_2_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: yolov8n.tf\assets


INFO:tensorflow:Assets written to: yolov8n.tf\assets
INFO:absl:Writing fingerprint to yolov8n.tf\fingerprint.pb


In [19]:
converter = tf.lite.TFLiteConverter.from_saved_model("yolov8n_tf")
# Enable TF Select Ops (keeps unsupported ops in the model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Default ops
    tf.lite.OpsSet.SELECT_TF_OPS      # Enables unsupported TensorFlow ops
]

tflite_model = converter.convert()
with open("yolov8n.tflite", "wb") as f:
    f.write(tflite_model)

print("Successfully converted to TFLite!")

Successfully converted to TFLite!


In [20]:
import os

# File path
file_path = "yolov8n.tflite"

# Get file size in MB
file_size = os.path.getsize(file_path) / (1024 * 1024)
print(f"File size: {file_size:.2f} MB")

File size: 12.19 MB


In [23]:
import os

# File path
file_path = "yolov8n.pt"

# Get file size in MB
file_size = os.path.getsize(file_path) / (1024 * 1024)
print(f"File size: {file_size:.2f} MB")

File size: 6.25 MB


In [25]:
# Optimizing the model
# Load the TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model("yolov8n_tf")

# Enable optimization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Default ops
    tf.lite.OpsSet.SELECT_TF_OPS      # Enables unsupported TensorFlow ops
]

# Convert to TFLite
tflite_model = converter.convert()

# Save optimized model
with open("yolov8n_optimized.tflite", "wb") as f:
    f.write(tflite_model)

print("Quantized TFLite model saved!")

Quantized TFLite model saved!


In [27]:
file_path = "yolov8n_optimized.tflite"

# Get file size in MB
file_size = os.path.getsize(file_path) / (1024 * 1024)
print(f"File size: {file_size:.2f} MB")

File size: 3.25 MB
